<a href="https://colab.research.google.com/github/lcroffi/CNN/blob/master/CNN_fully_trained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [0]:
img_width, img_height = 200, 200 

In [0]:
# Baixar as imagens do exame de citologia cervical
!wget -cq https://citologia-cervical.s3-sa-east-1.amazonaws.com/citologia.zip
!unzip -qq citologia.zip

In [0]:
from google.colab import drive
from keras.callbacks import *

drive.mount('/content/drive')
filepath="/content/drive/My Drive/Colab Notebooks/log/training:{epoch:03d}-val_acc:{val_acc:.3f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
def create_model(p, input_shape=(32, 32, 3)):
    # Initialising the CNN
    model = Sequential()
    # Convolution + Pooling Layer 
    model.add(Conv2D(32, (3, 3), padding='same', input_shape=input_shape, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Convolution + Pooling Layer 
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Convolution + Pooling Layer 
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Convolution + Pooling Layer 
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Flattening
    model.add(Flatten())
    # Fully connection
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(p))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(p/2))
    model.add(Dense(3, activation='softmax'))
    
    # Compiling the CNN
    optimizer = Adam(lr=1e-4)
    metrics=['accuracy']
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=metrics)
    return model

In [0]:
def run_training(bs=32, epochs=10):
    
    train_datagen = ImageDataGenerator(rescale = 1./255, 
                                       shear_range = 0.2, 
                                       zoom_range = 0.2, 
                                       horizontal_flip = True)
    test_datagen = ImageDataGenerator(rescale = 1./255)
 
    training_set = train_datagen.flow_from_directory('train',
                                                 target_size = (img_width, img_height),
                                                 batch_size = bs,
                                                 class_mode = 'categorical')
                                                 
    test_set = test_datagen.flow_from_directory('valid',
                                            target_size = (img_width, img_height),
                                            batch_size = bs,
                                            class_mode = 'categorical')
                                            
    model = create_model(p=0.6, input_shape=(img_width, img_height, 3))
    history = model.fit_generator(training_set,
                         steps_per_epoch=8000/bs,
                         epochs = epochs,
                         validation_data = test_set,
                         validation_steps = 2000/bs,
                         callbacks=callbacks_list)
    
    model.save("model.h5")
    
    score = model.evaluate_generator(test_set, 105)

    print ("%s: %.2f%%" % (model.metrics_names[0], score[0]*100))
    print ("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))
    
    return history

In [0]:
def history (hist):
  
    # listar todos os dados no history
    print(hist.history.keys())
    # Gráfico de treino - acurácia
    plt.plot(hist.history['acc'])
    plt.plot(hist.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    # Gráfico de treino - perda
    plt.plot(hist.history['loss'])
    plt.plot(hist.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()


In [0]:
def main():
    hist = run_training(bs=32, epochs=100)
    history(hist)
 
if __name__ == "__main__":
    main()

Found 423 images belonging to 3 classes.
Found 105 images belonging to 3 classes.
Epoch 1/100
250/250 [==============================] - 105s 419ms/step - loss: 1.0719 - acc: 0.3889 - val_loss: 1.0356 - val_acc: 0.3812

Epoch 00001: val_acc did not improve from 0.59066
Epoch 2/100
250/250 [==============================] - 99s 396ms/step - loss: 1.0397 - acc: 0.4186 - val_loss: 0.9896 - val_acc: 0.4848

Epoch 00002: val_acc did not improve from 0.59066
Epoch 3/100
250/250 [==============================] - 99s 396ms/step - loss: 1.0042 - acc: 0.4599 - val_loss: 0.9785 - val_acc: 0.5746

Epoch 00003: val_acc did not improve from 0.59066
Epoch 4/100
250/250 [==============================] - 99s 397ms/step - loss: 0.9442 - acc: 0.5349 - val_loss: 1.1009 - val_acc: 0.4945

Epoch 00004: val_acc did not improve from 0.59066
Epoch 5/100
250/250 [==============================] - 99s 397ms/step - loss: 0.8857 - acc: 0.5807 - val_loss: 1.0405 - val_acc: 0.5326

Epoch 00005: val_acc did not imp

In [0]:
from keras.models import load_model

model = load_model('model.h5')

In [0]:
!pip install --upgrade --quiet PyDrive
# para conectar com o Google Drive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

     |████████████████████████████████| 993kB 6.7MB/s 


In [0]:
# Importando imagens de teste do drive

link = 'https://drive.google.com/open?id=1sbjcK__NABa7gfsPOt7JM8jAaXCLCfKW'
fluff, id = link.split('=')

downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('teste1_cytHigh-grade Squamous Intraepithelial Lesion - 14659.jpg')

link = 'https://drive.google.com/open?id=1vDBjgozlaLg0tcGQ50b9wpOBz_NYyx8o'
fluff, id = link.split('=')

downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('teste2_cyt14721.jpg')

link = 'https://drive.google.com/open?id=14mCco19UM0k83Irdz3xk2lQaOs7a1YuK'
fluff, id = link.split('=')

downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('teste3_cytoCandida - 7557.jpg')

In [0]:
# Testando o modelo
from keras.preprocessing import image

test_image = image.load_img('teste3_cytoCandida - 7557.jpg', target_size = (200, 200))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)

if result[0][0] == 0:
    diagnostico = 'Carcinoma'
elif result[0][0] == 1:
    diagnostico = 'Normal'
else:
    diagnostico = 'Outros problemas'
    
print ('Diagnóstico:', diagnostico)

Diagnóstico: Carcinoma


In [0]:
# Best Weights 14/07

input_shape = (img_width, img_height, 3)
p = 0.6
bs = 32
# Initialising the CNN
model = Sequential()
# Convolution + Pooling Layer 
model.add(Conv2D(32, (3, 3), padding='same', input_shape=input_shape, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# Convolution + Pooling Layer 
model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# Convolution + Pooling Layer 
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# Convolution + Pooling Layer 
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
    
# Flattening
model.add(Flatten())
# Fully connection
model.add(Dense(64, activation='relu'))
model.add(Dropout(p))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dropout(p/2))
model.add(Dense(3, activation='softmax'))

model.load_weights('/content/drive/My Drive/Colab Notebooks/log/training:057-val_acc:0.656.hdf5') # Melhor treino do algoritmo

# Compiling the CNN
optimizer = Adam(lr=1e-4)
metrics=['accuracy']
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=metrics)

test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('valid',
                                            target_size = (img_width, img_height),
                                            batch_size = bs,
                                            class_mode = 'categorical')

score = model.evaluate_generator(test_set, 105)

print ("%s: %.2f%%" % (model.metrics_names[0], score[0]*100))
print ("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

Found 105 images belonging to 3 classes.
loss: 280.88%
acc: 65.71%
